# Linear Web Crawls with Dates

In this section, we will look at using the `datetime` modules to create many urls that are based on dates.

## The Problem

* We have to create many urls based on dates/time.
    * Example: `http://www.thecurrent.org/playlist/2014-01-01/01`
* Working with dates is tough
    * different days per month
    * leap years

## The solution

We can use the `datetime` module to

* Create many dates
* Print to a specific output

In [1]:
import datetime

## Example - Scraping The Current

In a previous lab, we wrote a function get the information for a single page (1 hour of songs).  Here is the resulting functions.

In [2]:
import requests
from bs4 import BeautifulSoup

def get_period(soup):
    search = soup('span', class_="hour-header open")
    if len(search) > 0:
        return search[0].next.split('to')[0].rstrip()[-2:]
    else:
        return None

def get_day(soup):
    search = soup('a', class_="start-picker")
    if len(search) > 0:
        return search[0].next.split(',')[0]
    else:
        return None
    
def get_song_info(url):
    print("Starting {0} urls".format(url))
    date = url.split('/')[-2]
    s = requests.Session()
    r = s.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    period = get_period(soup)
    day_of_week = get_day(soup)
    soup = BeautifulSoup(r.content)
    titles = [t.next.strip() for t in soup.findAll('h5', class_="title")]
    artists = [a.next.strip() for a in soup.findAll('h5',class_='artist')]
    times = [d.time.next.strip() for d in soup('div', class_="two columns songTime")]
    song_info = [(day_of_week, date, time, period, title, artist) 
             for time, title, artist in zip(times, titles, artists)]
    return song_info

# Collecting a weeks worth of music

## Step 1 - Identify the url pattern

The Current uses urls of the following pattern

    'http://www.thecurrent.org/playlist/2017-05-04/10'

or 

    'http://www.thecurrent.org/playlist/year-month-day/hour'

## Question: How might you generate all combinations for a given year?

**Answer:** Python has a tool for that!

In [3]:
numdays = 7
# Set the base url as today
base = datetime.datetime.today()
# Make new times by subracting an hour at a time.
dts = [base - datetime.timedelta(hours = h) for h in range(0, numdays*24)]
dts

[datetime.datetime(2020, 10, 27, 11, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 10, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 9, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 8, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 7, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 6, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 5, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 4, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 3, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 2, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 1, 38, 18, 783062),
 datetime.datetime(2020, 10, 27, 0, 38, 18, 783062),
 datetime.datetime(2020, 10, 26, 23, 38, 18, 783062),
 datetime.datetime(2020, 10, 26, 22, 38, 18, 783062),
 datetime.datetime(2020, 10, 26, 21, 38, 18, 783062),
 datetime.datetime(2020, 10, 26, 20, 38, 18, 783062),
 datetime.datetime(2020, 10, 26, 19, 38, 18, 783062),
 datetime.datetime(2020, 10, 26, 18, 38, 18, 783062),
 datetime.datetime(2020, 10, 26, 17, 3

## Outputing the time in a specific format.

* datetime objects have a `strftime` method for outputing formatted dates
    * `%Y` is the year in 4 digits
    * `%m` is the month in 2 digits
    * etc.
* See [this site](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior) for more information.

In [4]:
fmt = 'http://www.thecurrent.org/playlist/%Y-%m-%d/%H'

date = datetime.datetime(2000,1,1,1)
date.strftime(fmt)

'http://www.thecurrent.org/playlist/2000-01-01/01'

## Let's make some functions

In [5]:
def output_address(dt):
    fmt = 'http://www.thecurrent.org/playlist/%Y-%m-%d/%H'
    return dt.strftime(fmt)

def test_output_address():
    date = datetime.datetime(2000,1,1,1)
    assert output_address(date) == 'http://www.thecurrent.org/playlist/2000-01-01/01'
test_output_address()

In [6]:
urls = [output_address(d) for d in dts]
urls[:10]

['http://www.thecurrent.org/playlist/2020-10-27/11',
 'http://www.thecurrent.org/playlist/2020-10-27/10',
 'http://www.thecurrent.org/playlist/2020-10-27/09',
 'http://www.thecurrent.org/playlist/2020-10-27/08',
 'http://www.thecurrent.org/playlist/2020-10-27/07',
 'http://www.thecurrent.org/playlist/2020-10-27/06',
 'http://www.thecurrent.org/playlist/2020-10-27/05',
 'http://www.thecurrent.org/playlist/2020-10-27/04',
 'http://www.thecurrent.org/playlist/2020-10-27/03',
 'http://www.thecurrent.org/playlist/2020-10-27/02']

In [7]:

info_lists = [get_song_info(url) for url in urls]
flat_info = [row for info_list in info_lists for row in info_list]
lines = [",".join(row) for row in flat_info]

with open('the_current_last_year.csv', 'w') as outfile:
    header = "Weekday,Date,Time,Period,Song_Title,Artist"
    print(header, file=outfile)
    count = 0
    for url in urls:
        for line in lines:
            count += 1
            if count % 5 == 0:
                print("Processed {0} songs".format(count))
            print(line, file=outfile)

Starting http://www.thecurrent.org/playlist/2020-10-27/11 urls
Starting http://www.thecurrent.org/playlist/2020-10-27/10 urls
Starting http://www.thecurrent.org/playlist/2020-10-27/09 urls
Starting http://www.thecurrent.org/playlist/2020-10-27/08 urls
Starting http://www.thecurrent.org/playlist/2020-10-27/07 urls


KeyboardInterrupt: 